In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import random
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn import metrics

# MINIMAX ALGORITHM
This is a reinforcement model which does not use a training dataset and learns based on its experience.
Hence, this was our first choice.

In [ ]:
#Use the following nine numbers to indicate the position of the board
# 0 1 2
# 3 4 5
# 6 7 8

# method of winning
WINNING_TRIADS = ((0, 1, 2), (3, 4, 5), (6, 7, 8),
                  (0, 3, 6), (1, 4, 7),(2, 5, 8),
                  (0, 4, 8), (2, 4, 6))

PRINTING_TRIADS = ((0, 1, 2), (3, 4, 5), (6, 7, 8))

# experss with a one-dimensional list
SLOTS = (0, 1, 2, 3, 4, 5, 6, 7, 8)

# -1 X player.  0 blank.  1 O player
X_token = -1
Open_token = 0
O_token = 1

MARKERS = ['_', 'O', 'X']
END_PHRASE = ('tie', 'win', 'lose')


def alpha_beta_valuation(board, player, next_player, alpha, beta):
    #AlphaBeta [-1,0,1] 
    # calculate the board now with AlphaBeta pruning
    wnnr = winner(board)
    if wnnr != Open_token:
        # one player win
        return wnnr
    elif not legal_move_left(board):
        # no blank and tie
        return 0
    
    # check all the avilable poisiton for player
    for move in SLOTS:
        if board[move] == Open_token:
            board[move] = player
            
            # change to the other player
            val = alpha_beta_valuation(board, next_player, player, alpha, beta)
            board[move] = Open_token
            if player == O_token:  # O player, chose max method
                if val > alpha:
                    alpha = val
                if alpha >= beta:
                    return beta  # return the max value and pruning
            else:  #X player min method
                if val < beta:
                    beta = val
                if beta <= alpha:
                    return alpha  # return the min value and pruning
    if player == O_token:
        retval = alpha
    else:
        retval = beta
    return retval


def print_board(board):
    # print board
    for row in PRINTING_TRIADS:
        r = ' '
        for hole in row:
            r += MARKERS[board[hole]] + ' '
        print(r)


def legal_move_left(board):
    # check is there some avilable position
    for slot in SLOTS:
        if board[slot] == Open_token:
            return True
    return False


def winner(board):
    # return -1 means x win , 1 means O win , 0 means tie and not the end
    for triad in WINNING_TRIADS:
        triad_sum = board[triad[0]] + board[triad[1]] + board[triad[2]]
        if triad_sum == 3 or triad_sum == -3:
            return board[triad[0]]  # 1:X,1:O
    return 0


def determine_move(board):
    # find out the which is the best move for O player. if all the results are equal, chose randomly
    best_val = -2  #[-1,0,1]
    my_moves = []
    print("star thinking")
    for move in SLOTS:
        if board[move] == Open_token:
            board[move] = O_token
            val = alpha_beta_valuation(board, X_token, O_token, -2, 2)
            board[move] = Open_token
            print("if Computer choose", move, ",lead to", END_PHRASE[val])
            if val > best_val:
                best_val = val
                my_moves = [move]
            if val == best_val:
                my_moves.append(move)
    return random.choice(my_moves)


HUMAN = 1
COMPUTER = 0


def main():
    next_move = HUMAN
    opt = input("please enter，X means palyer，O means computer：")
    if opt == "X":
        next_move = HUMAN
    elif opt == "O":
        next_move = COMPUTER
    else:
        print("enter mistake，palyer first")

    board = [Open_token for i in range(9)]

    while legal_move_left(board) and winner(board) == Open_token:
        print()
        print_board(board)
        if next_move == HUMAN and legal_move_left(board):
            try:
                humanmv = int(input("choose the place u want(0-8)："))
                if board[humanmv] != Open_token:
                    continue
                board[humanmv] = X_token
                next_move = COMPUTER
            except:
                print("enter mistake，plz try again")
                continue
        if next_move == COMPUTER and legal_move_left(board):
            mymv = determine_move(board)
            print("Computer decision", mymv)
            board[mymv] = O_token
            next_move = HUMAN

    print_board(board)
    print(["tie", "Computer wins", "you win"][winner(board)])


if __name__ == '__main__':
    main()

However, what was expected in the project was a machine learning algorithm which predicts based on the taining dataset.

# DECISION TREE ALGORITHM
We decided to use the decision tree classifier which will branch out each move into a number of possibilities 
and predict the best move based on the dataset.

In [ ]:
df = pd.read_csv('tic-tac-toe-endgame.csv')
cl = df["V10"].unique()

x=df.drop("V10", axis=1)
y = df["V10"]

x = x.apply(preprocessing.LabelEncoder().fit_transform)
le= preprocessing.LabelEncoder()
y = le.fit_transform(y)

x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=0.29,random_state=0)

clf = DecisionTreeClassifier(criterion='entropy', max_depth=9, min_impurity_decrease=0.0, min_samples_leaf=1, class_weight="balanced", splitter='best')

clf = clf.fit(x_train, y_train)

result = clf.score(x_train,y_train)
print("score for train set:",result)
result1 = clf.score(x_test,y_test)
print("score for test set:",result1)

board = np.array([1,1,2,0,0,2,0,0,0])
#X=1 and O=2
#values_filled = (board == 0).sum()
#print(values_filled)
    
df_possibilities = x.loc[
x['V1'].apply(lambda x: range(0,8) if board[0]==0 else x==board[0])&
x['V2'].apply(lambda x: range(0,8) if board[1]==0 else x==board[1])&
x['V3'].apply(lambda x: range(0,8) if board[2]==0 else x==board[2])&
x['V4'].apply(lambda x: range(0,8) if board[3]==0 else x==board[3])&
x['V5'].apply(lambda x: range(0,8) if board[4]==0 else x==board[4])&
x['V6'].apply(lambda x: range(0,8) if board[5]==0 else x==board[5])&
x['V7'].apply(lambda x: range(0,8) if board[6]==0 else x==board[6])&
x['V8'].apply(lambda x: range(0,8) if board[7]==0 else x==board[7])&
x['V9'].apply(lambda x: range(0,8) if board[8]==0 else x==board[8])
]

                              
df_possibilities = df_possibilities.reset_index(drop=True)
prediction = clf.predict(df_possibilities)
prediction = pd.DataFrame(prediction)
prediction.columns = ["prediction"]
df_possibilities_pred = pd.concat([df_possibilities,prediction],axis=1)

df_win = df_possibilities_pred.loc[df_possibilities_pred['prediction'] == 1] #POSITIVES
df_lose = df_possibilities_pred.loc[df_possibilities_pred['prediction'] == 0] #NEGATIVES
    
empty = []
filled = []
a = board.tolist()
for i in range(len(a)):
    if a[i] == 0:
        empty.append(i)
    elif a[i] != 0:
        filled.append(i)
        
def getlistnum(li):
    li = list(li)
    set1 = set(li)
    dict1 = {}
    for item in set1:
        dict1.update({item:li.count(item)})
    return dict1

winning_pos = []
losing_pos = []  
    
if a.count(2) >= a.count(1): 
    for i in empty:
        winning = getlistnum(df_win.iloc[:,i]).get(2,0)/len(df_win)
        losing = getlistnum(df_lose.iloc[:,i]).get(2,0)/len(df_lose)
        winning_pos.append(winning)
        losing_pos.append(losing)
    #Generate a random number and assign it to randInt
        
    
com = pd.concat([pd.DataFrame(winning_pos),pd.DataFrame(losing_pos)],axis=1)
com.columns = ["winning","losing"]
com["win"] = np.where(com["winning"]>com["losing"],1,0)
best_move_index = com.loc[com["win"]==1]["winning"].idxmax()
predicted_position = empty[best_move_index]
print (predicted_position)

In [ ]:
cm = clf.predict(x_test)
print("Accuracy: {0:.1f}%".format(metrics.accuracy_score(y_test,cm) * 100))

In [ ]:
plt.figure(figsize=(4,4))
sns.heatmap(metrics.confusion_matrix(y_test,cm), annot=True, linewidths=.5, square = True, cmap = 'Blues_r');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
plt.show()

Although the model offers a accuracy of 94% it has certain limitations

### LIMITATIONS OF THE ALGORITHM
- The dataset is divided into "Winning" and "Losing" moves. There is no tie bifurcation
- As a result, when the algorithm knows it is going to be a tie - it does not predict a move
- It does not predict the first move and the last move
- All its predictions are limited to data provided and it is not learning based on experiences. That said, it does not predict when the player makes a move that is not a part of the dataset.

There are numerous possibilities to the game and adding all the possibilities to your dataset is not practical.

# WHAT WE FINALLY DID...
- We planned on using the traditional method where we pre-defined rules for the AI to play
- We defined each fuction that helped the smooth functioning of the game

In [ ]:
## PREDEFINED RULES
 
def drawBoard(board):
    print('\n\n\n\n')
    print('\t\t\t┌─┬─┬─┐')
    print('\t\t\t│'+board[1]+'│'+board[2]+'│'+board[3]+'│')
    print('\t\t\t├─┼─┼─┤')
    print('\t\t\t│'+board[4]+'│'+board[5]+'│'+board[6]+'│')
    print('\t\t\t├─┼─┼─┤')
    print('\t\t\t│'+board[7]+'│'+board[8]+'│'+board[9]+'│')
    print('\t\t\t└─┴─┴─┘')

 
 
def inputPlayerLetter():
    #Let the player choose X or O
    #Return a list that contains the choice of player and the choice of computer
    letter = ''
    while not (letter == 'X' or letter == 'O'):
        print('Do you want to be X or O?')
        letter = input().upper()
 
    if letter == 'X':
        return ['X', 'O']
    else:
        return ['O', 'X']
 
 
def whoGoesFirst():
    #Randomly decide who goes first
    if random.randint(0, 1) == 0:
        return 'computer'
    else:
        return 'player'
 
 
def playAgain():
    #Ask the human whether he wants to play again
    print('Do you want to play again? (yes or no)')
    return input().lower().startswith('y')
 
def makeMove(board, letter, move):
    #make the move
    board[move] = letter
 
 
def isWinner(bo, le):
    #check whether there is a winner
    return ((bo[7] == le and bo[8] == le and bo[9] == le) or
            (bo[4] == le and bo[5] == le and bo[6] == le) or
            (bo[1] == le and bo[2] == le and bo[3] == le) or
            (bo[7] == le and bo[4] == le and bo[1] == le) or
            (bo[8] == le and bo[5] == le and bo[2] == le) or
            (bo[9] == le and bo[6] == le and bo[3] == le) or
            (bo[7] == le and bo[5] == le and bo[3] == le) or
            (bo[9] == le and bo[5] == le and bo[1] == le))
 
def getBoardCopy(board):
    dupeBoard = []
    for i in board:
        dupeBoard.append(i)
    return dupeBoard
 
def isSpaceFree(board, move):
    # judge if the position is empty 
    return board[move] == ' '
 
def getPlayerMove(board):
    move = ' '
    while move not in '1 2 3 4 5 6 7 8 9'.split() or not isSpaceFree(board, int(move)):
        print('What is your next move? (1-9)')
        move = input()
    return int(move)
 
def chooseRandomMoveFromList(board, movesList):
    possibleMoves = []
    for i in movesList:
        if isSpaceFree(board, i):
            possibleMoves.append(i)
 
    if len(possibleMoves) != 0:
        return random.choice(possibleMoves)
    else:
        return None

    
def getComputerMove(board, computerLetter):
    if computerLetter == 'X':
        playerLetter = 'O'
    else:
        playerLetter = 'X'
 
 
    # Tic Tac Toe AI algoritmn:
    #First judge whether computer can directly win in the next move, if so, move to that position
    for i in range(1, 10):
        copy = getBoardCopy(board)
        if isSpaceFree(copy, i):
            makeMove(copy, computerLetter, i)
            if isWinner(copy, computerLetter):
                return i
 
 
    #Judge whether the opponent can win in the next step, if so, block this position
    for i in range(1, 10):
        copy = getBoardCopy(board)
        if isSpaceFree(copy, i):
            makeMove(copy, playerLetter, i)
            if isWinner(copy, playerLetter):
                return i

    #if four corners have space, choose one of them
    move = chooseRandomMoveFromList(board, [1, 3, 7, 9])
    if move != None:
        return move
 
 
    # if center is empty, choose the empty
    if isSpaceFree(board, 5):
        return 5
 
 
    # choose one of four sides
    return chooseRandomMoveFromList(board, [2, 4, 6, 8])
 

def isBoardFull(board):
    for i in range(1, 10):
        if isSpaceFree(board, i):
            return False
    return True
 
 
 
print('Welcome to Tic Tac Toe!')
 
while True:
    # Update the board
    theBoard = [' '] * 10
    playerLetter, computerLetter = inputPlayerLetter()
    turn = whoGoesFirst()
    print('The ' + turn + ' will go first.')
    gameIsPlaying = True
 
 
    while gameIsPlaying:
        if turn == 'player':
            # Human's turn
            drawBoard(theBoard)
            move = getPlayerMove(theBoard)
            makeMove(theBoard, playerLetter, move)
 
 
            if isWinner(theBoard, playerLetter):
                drawBoard(theBoard)
                print('Hooray! You have won the game!')
                gameIsPlaying = False
            else:
                if isBoardFull(theBoard):
                    drawBoard(theBoard)
                    print('The game is a tie!')
                    break
                else:
                    turn = 'computer'
 
 
        else:# Computer's turn
            move = getComputerMove(theBoard, computerLetter)
            makeMove(theBoard, computerLetter, move)
 
 
            if isWinner(theBoard, computerLetter):
                drawBoard(theBoard)
                print('The computer has beaten you! You lose.')
                gameIsPlaying = False
            else:
                if isBoardFull(theBoard):
                    drawBoard(theBoard)
                    print('The game is a tie!')
                    break
                else:
                    turn = 'player'
 
 
    if not playAgain():
        break